In [6]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup

In [7]:
url = 'http://www.finances.gov.tn/fr/faq?body_value=&field_themef_target_id=All&page=0'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

# Find the last page button and extract the URL
last_page_url  = soup.find('a', attrs={"title": "Aller à la dernière page"}).get('href')

# Extract the maximum page number from the last page URL
max_pages = int(last_page_url.split('=')[-1]) + 1 

In [8]:
questions_list=[]
answers_list=[]

for page in range(max_pages):
    
    page_url = f'http://www.finances.gov.tn/fr/faq?body_value=&field_themef_target_id=All&page={page}'
    response = requests.get(page_url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    questions = soup.find_all(attrs={"class": "question"})
    answers = soup.find_all(attrs={"class": "reponse"})
    questions = [question.text.strip() for question in questions]
    answers = [answer.text.strip() for answer in answers]
    
    questions_list.append(questions)
    answers_list.append(answers)
    
# Flatten the lists
questions_list = [item for sublist in questions_list for item in sublist]
answers_list = [item for sublist in answers_list for item in sublist]

In [9]:
QA_dict = {'questions':questions_list, 'answers':answers_list}
QA_df = pd.DataFrame(QA_dict)
QA_df.head()

,questions,answers
0,Est-ce qu’une personne physique soumise à l’im...,Les personnes physiques soumises à l'impôt sur...
1,Est-ce que les rémunérations du gérant d’une s...,Les rémunérations du gérant d'une SUARL sont d...
2,Si l’employeur refuse l’application de l’artic...,La déduction aura lieu du revenu global lors d...
3,Quel est le taux de la retenue à la source à a...,Les montants égaux ou supérieurs à 1000 D TTC ...
4,Est-ce que les associations sont soumises à l’...,Les associations sont hors champ d'application...


In [12]:
# Saving into QA_finances.gov.tn.csv file
QA_df.to_csv('QA1.csv', index=False)